In [3]:
import pandas as pd
import numpy as np
import sqlalchemy as sqla
from io import StringIO

#establish connection to database
conn = sqla.create_engine('sqlite:///yelp.sqlite')

#split business.json into bussines, attributes, hours and store in a database 
business=pd.read_json("/Users/sihuili/Downloads/yelp_dataset/yelp_academic_dataset_business.json",lines=True)
business.set_index("business_id",inplace=True)

attributes=pd.DataFrame.from_dict(business["attributes"].dropna().to_dict(),orient="index")
attributes.index.name="business_id"

hours=pd.DataFrame.from_dict(business["hours"].dropna().to_dict(),orient="index")
hours.index.name="business_id"

business.drop(["hours","attributes"],axis=1,inplace=True)

business.to_sql('business',conn,if_exists="replace")
hours.to_sql('hours',conn,if_exists="replace")
attributes.to_sql('attributes',conn,if_exists="replace")

#convert checkin.json into clean and neat dataframe and store in the database
checkins=pd.read_json("/Users/sihuili/Downloads/yelp_dataset/yelp_academic_dataset_checkin.json",lines=True,orient="record",chunksize=10000)
for checkin in checkins:
    checkin.set_index("business_id",inplace=True)
    checkin=pd.DataFrame.from_dict(checkin["time"].to_dict(),orient="index")
    checkin.reset_index(inplace=True)
    checkin=checkin.melt(id_vars="index")
    days=pd.read_table(StringIO(checkin['variable'].to_csv(None,index=None)),sep='-',header=None)
    checkin=checkin.join(days)
    checkin.drop("variable",axis=1,inplace=True)
    checkin=checkin[["index",0,1,"value"]]
    checkin.columns=["business_id","weekday","hour","checkins"]
    checkin.set_index("business_id",inplace=True)
    checkin["weekday"]=checkin["weekday"].astype("category")
    checkin.to_sql('checkin',conn,if_exists="append")

#store review.json into the database
reviews=pd.read_json("/Users/sihuili/Downloads/yelp_dataset/yelp_academic_dataset_review.json",chunksize=100000,lines=True)
for review in reviews:
    review.set_index("business_id",inplace=True)
    review.to_sql("review",conn,if_exists="append")

In [64]:
checkin.head()

,weekday,hour,checkins
business_id,,,
--EF5N7P70J_UYBTPypYlA,Sun,0,NaN
-0CCHBui57tZ_1y_14X-5Q,Sun,0,NaN
-0DET7VdEQOJVJ_v6klEug,Sun,0,5.0
-0K4W5p3wbQe_vDoB5jMgQ,Sun,0,1.0
-0RU0vASJOV383dYA8VZgA,Sun,0,1.0


In [65]:
checkin.shape

(1188600, 3)

In [71]:
import sqlite3
conn = sqlite3.connect('yelp.sqlite')
c = conn.cursor()

In [113]:
cursor = c.execute(''' Select * from checkin; ''')
business_id = []
day_of_week = []
checkin_hours = []
check_in_counts = []
for i in cursor:
    business_id.append(i[0])
    day_of_week.append(i[1])
    hours.append(i[2])
    check_in_counts.append(i[3])

In [114]:
checkin_df = pd.DataFrame({'business_id':business_id, 'day_of_week':day_of_week, 'hours':hours, 'check_in_counts':check_in_counts})

In [116]:
checkin_df.head()

,business_id,day_of_week,hours,check_in_counts
0,--cZ6Hhc9F7VkKXxHMVZSQ,Fri,0,18.0
1,--cgVkbWTiga3OYTkymKqA,Fri,0,NaN
2,--i1tTcggBi4cPkd-h5hDg,Fri,0,NaN
3,--lZAZSwpP_axKoL4lR9dQ,Fri,0,NaN
4,-06-Fdi30wJx-JA8P7CAng,Fri,0,2.0


In [ ]:
checkins=pd.read_json("/Users/sihuili/Downloads/yelp_dataset/yelp_academic_dataset_checkin.json",lines=True,orient="record",chunksize=10000)

In [117]:
cursor = c.execute(''' Select * from business limit 100; ''')
for i in cursor:
    print(i)

('Apn5Q_b6Nz61Tq4XzPdf9A', '1314 44 Avenue NE', 'Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel', 'Calgary', 1, 51.0918130155, -114.031674872, 'Minhas Micro Brewery', '', 'T2E 6L6', 24, 4.0, 'AB')
('AjEbIBw6ZFfln7ePHha9PA', '', 'Chicken Wings, Burgers, Caterers, Street Vendors, Barbeque, Food Trucks, Food, Restaurants, Event Planning & Services', 'Henderson', 0, 35.9607337, -114.939821, "CK'S BBQ & Catering", '', '89002', 3, 4.5, 'NV')
('O8S5hYJ1SMc8fA4QBtVujA', '1335 rue Beaubien E', 'Breakfast & Brunch, Restaurants, French, Sandwiches, Cafes', 'Montréal', 0, 45.5405031, -73.5993003, 'La Bastringue', 'Rosemont-La Petite-Patrie', 'H2G 1K7', 5, 4.0, 'QC')
('bFzdJJ3wp3PZssNEsyU23g', '211 W Monroe St', 'Insurance, Financial Services', 'Phoenix', 1, 33.4499993, -112.0769793, 'Geico Insurance', '', '85003', 8, 1.5, 'AZ')
('8USyCYqpScwiNEb58Bt6CA', '2005 Alyth Place SE', 'Home & Garden, Nurseries & Gardening, Shopping, Local Services, Automotive, Electronics Repair', 'Calgary', 

In [120]:
i

('od5dCR8QPU67t8Ku6CBezw',
 '444 W Broadway Rd',
 'Financial Services, Banks & Credit Unions',
 'Tempe',
 1,
 33.407784,
 -111.945686,
 'Chase Bank',
 '',
 '85282',
 5,
 2.5,
 'AZ')

In [ ]:
cursor = c.execute(''' Select * from business limit 100; ''')
for i in cursor:
    print(i)

In [122]:
business.head()

,address,categories,city,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
business_id,,,,,,,,,,,,
Apn5Q_b6Nz61Tq4XzPdf9A,1314 44 Avenue NE,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
AjEbIBw6ZFfln7ePHha9PA,,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
O8S5hYJ1SMc8fA4QBtVujA,1335 rue Beaubien E,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
bFzdJJ3wp3PZssNEsyU23g,211 W Monroe St,"Insurance, Financial Services",Phoenix,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
8USyCYqpScwiNEb58Bt6CA,2005 Alyth Place SE,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


In [126]:
attributes.head()

,BikeParking,BusinessAcceptsCreditCards,BusinessParking,GoodForKids,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,...,Music,Smoking,ByAppointmentOnly,AcceptsInsurance,BusinessAcceptsBitcoin,HairSpecializesIn,AgesAllowed,RestaurantsCounterService,Open24Hours,DietaryRestrictions
business_id,,,,,,,,,,,,,,,,,,,,,
--1UhMGODdWsrMastO9DZw,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",True,NaN,quiet,True,casual,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--6MefnULPED_I942VcFNA,False,True,"{'garage': False, 'street': False, 'validated'...",True,False,average,False,casual,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--7zmmkVg-IMGaXbuVd0SQ,True,True,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--8LPVSo5i0Oo61X01sV9A,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN
--9QQLMTbFzLJ_oT-ON3Xw,NaN,True,"{'garage': False, 'street': False, 'validated'...",True,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
hours=pd.DataFrame.from_dict(business["hours"].dropna().to_dict(),orient="index")
hours.index.name="business_id"

In [130]:
hours.head()

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
business_id,,,,,,,
0,8:30-17:0,11:0-21:0,11:0-21:0,11:0-21:0,11:0-21:0,11:0-21:0,NaN
1,NaN,NaN,NaN,NaN,17:0-23:0,17:0-23:0,17:0-23:0
2,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0
4,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,NaN,NaN
5,5:30-20:0,5:30-20:0,5:30-20:0,5:30-20:0,5:30-21:0,5:30-21:0,6:30-19:0


In [131]:
business.shape

(188593, 15)

In [139]:
city = business.city.value_counts().index

In [133]:
df = business.groupby('business_id')['business_id'].nunique()

In [136]:
df.value_counts()

1    188593
Name: business_id, dtype: int64

In [176]:
D_city = []
for i in list(city):
    if len(i) > 0:
        if i[0:2] == 'La':
            D_city.append(i)

In [181]:
D_city.sort()
D_city

['La Baie',
 'La Prairie',
 'La Salle',
 'La Vegas',
 'LaSalle',
 'Lachenaie',
 'Lachine',
 'Lachute',
 'Lacolle',
 'Lagrange',
 'Lake Las Vegas',
 'Lake Norman',
 'Lake Park',
 'Lake Wylie',
 'Lakewood',
 'Lakewood, Oh',
 'Lam',
 'Lambton',
 'Landau',
 'Landsberg',
 'Larimer',
 'Las  Vegas',
 'Las Vegas',
 'Las Vegas & Henderson',
 'Las Vegas East',
 'Las Vegas Nevada',
 'Las Vegas Nv',
 'Las Vegas,',
 'Las Vegas, NV',
 'Las Vegass',
 'Las vegas',
 'LasVegas',
 'Lasalle',
 'Laval',
 'Laval, Pont Viau',
 'Laval, Ste Dorothee',
 'Lavaltrie',
 'Laveen',
 'Laveen Village',
 'Lawrence',
 'Lawrenceville']

In [182]:
% pwd

'/Users/sihuili'